In [30]:
import nltk
import re
import pickle
from sklearn.datasets import load_files
from nltk.stem.porter import *

In [31]:
def simplify(doc):
    #Remove all other letters except a-zA-Z and spaces. And make it lower case
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A).lower().strip() 
    # Tokenize
    tokens = nltk.WordPunctTokenizer().tokenize(doc) 
    # Remove stopwords
    filtered_tokens = [token for token in tokens if token not in nltk.corpus.stopwords.words('english')] 
    # Re-create document from filtered tokens
    doc = ' '.join(filtered_tokens) 
    return doc

In [32]:
def get_words(sentence):
    stemmer = PorterStemmer()
    words = [stemmer.stem(x) for x in simplify(sentence).split()]
    return words

In [33]:
#dataset = load_files('reviews')
#features, labels = dataset.data, dataset.target

In [34]:
# with open('features.pickle', 'wb') as f:
#     pickle.dump(features, f)
# with open('labels.pickle', 'wb') as f:
#     pickle.dump(labels, f)

In [35]:
# Loading dataset (Unpickling datasets)
with open('features.pickle', 'rb') as f:
    features = pickle.load(f)
    
with open('labels.pickle', 'rb') as f:
    labels = pickle.load(f)

In [36]:
word_sentiment = {}

In [37]:
for i in range(len(features)):
    sentiment, line = labels[i],str(features[i])
    words = get_words(line)
    for word in words:
        try: 
            # Setting word sentiment weights. 
            if int(sentiment) == 1:
                word_sentiment[word][0] = word_sentiment[word][0] + 1 
            else:
                word_sentiment[word][1] = word_sentiment[word][1] + 1 
        except: 
            # New word. Create new entry
            if int(sentiment) == 1:
                word_sentiment[word] = [1, 0, 0]  
            else:
                word_sentiment[word] = [0, 1 ,0]
                
        finally: 
            # Increce occurrences count of words.
            word_sentiment[word][2] += 1

In [38]:
# Weighted sum of the sentiment of each word (divided by the number of occurences of the word)

word_weighted_sentiment = {word: ( (word_sentiment[word][0]-word_sentiment[word][1]) / word_sentiment[word][2] ) for word in word_sentiment.keys()}


In [44]:
text_to_sentiment = "The frivolous blend of ludicrous mystery and tepid romance offered in Digging For Fire got me quickly bored."
#"Despite the flaw of being heavily biased in portraying [Abdul] El-Sayed in the best possible light, How to Fix a Primary does an admirable job of putting his campaign in a larger context of how much can be bought and sold in an election."


In [45]:
sentiment = 0
words = get_words(text_to_sentiment)
for word in words:
    try:
        # Compute sum of weighted sentiments of the words in the review
        sentiment += word_weighted_sentiment[word] 
    except:
        # For new words sentiment is 0 
        sentiment += 0 
if sentiment >= 0:
    print("Possitive")
else:
    print("Negative")

Negative
